I can use a district data of Thailand which contains latitude and longtitude of each district to cluster the area and count number of place like this using Foursquare API which scope data in some categories.

In [1]:
import numpy as np
import pandas as pd
import requests
import json
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

This is data which contains data of sub-district location in Thailand (I already translate columns name to English)

In [2]:
df=pd.read_excel("Thailand_location.xlsx")
df.head()

,Sub-district,District_ID,District,Province_ID,Province,LAT,LONG
0,Ko Sarai,9101,Mueang Satun,91,Satun,6.546,99.706
1,Maptaphut,2101,Mueang Rayong,21,Rayong,12.646,101.171
2,Maptaphut,2101,Mueang Rayong,21,Rayong,12.645,101.170
3,Maptaphut,2101,Mueang Rayong,21,Rayong,12.649,101.174
4,Maptaphut,2101,Mueang Rayong,21,Rayong,12.644,101.169


I had to filter it to contains only district in Bangkok and made an assumption that mean of latitude and longitude is the center of each district

In [3]:
df_usable=df[df["Province"]=="Bangkok"].groupby("District").mean().reset_index()
del df_usable['Province_ID']
df_usable

,District,District_ID,LAT,LONG
0,Bang Bon,1050,13.646000,100.370000
1,Bang Kapi,1006,13.771000,100.647500
2,Bang Khae,1040,13.710500,100.397500
3,Bang Khen,1005,13.867000,100.628000
4,Bang Kho Laem,1031,13.698667,100.506000
5,Bang Khun Thian,1021,13.580000,100.414500
6,Bang Na,1047,13.672000,100.616000
7,Bang Phlat,1025,13.787000,100.493250
8,Bang Rak,1004,13.728000,100.523200
9,Bang Su,1029,13.820000,100.529000


Start getting data from Foursquare API

In [4]:
# @hidden_cell
#credentials
CLIENT_ID = 'QKXUD0JQ2LLSHHFHDMOMVCMNIGPNASP5R1F03IU1DPLZ3IXB' # your Foursquare ID
CLIENT_SECRET = 'GB3ZAQIVAROSJCRJ3SLSPYCNA4KSJ2R1F4BKIVFP4U5IQVHC' # your Foursquare Secret
VERSION = '20200714' # Foursquare API version

In [5]:
neighborhood_latitude = df_usable.loc[0, 'LAT'] # neighborhood latitude value
neighborhood_longitude = df_usable.loc[0, 'LONG'] # neighborhood longitude value

neighborhood_name = df_usable.loc[0, 'District'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Bang Bon are 13.646, 100.37.


In [6]:
LIMIT=100
radius = 500

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['District', 
                  'District Latitude', 
                  'District Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [7]:
bangkok_venues = getNearbyVenues(names=df_usable['District'],
                                   latitudes=df_usable['LAT'],
                                   longitudes=df_usable['LONG']
                                  )

Bang Bon
Bang Kapi
Bang Khae
Bang Khen
Bang Kho Laem
Bang Khun Thian
Bang Na
Bang Phlat
Bang Rak
Bang Su
Bangkok Noi
Bangkok Yai
Bung Kum
Chatuchak
Chom Thong
Din Daeng
Don Mueang
Dusit
Huai Khwang
Khanna Yao
Khlong Sam Wa
Khlong San
Khlong Toei
Lak Si
Lat Krabang
Lat Phrao
Min Buri
Nong Chok
Nong Khaem
Pathum Wan
Phasi Charoen
Phaya Thai
Phra Khanong
Phra Nakhon
Pom Prap Sattruphai
Prawet
Rat Burana
Rat Thewi
Sa Thon
Sai Mai
Samphanthawong
Saphan Sung
Suan Luang
Taling Chan
Thawi Watthana
Thon Buri
Thung Khu
Wang Thong Lang
Watthana
Yannawa


In [8]:
print(bangkok_venues.shape)
bangkok_venues.head()

(962, 7)


,District,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bang Bon,13.646,100.3700,PUILIN KITCHEN HongKong Food Style,13.643194,100.366970,Asian Restaurant
1,Bang Kapi,13.771,100.6475,ตลาดนัดแฟลตคลองจั่น,13.770105,100.649124,Flea Market
2,Bang Kapi,13.771,100.6475,ร้านข้าวแกงครัวปักษ์ใต้คลองจั่น,13.770482,100.648638,Thai Restaurant
3,Bang Kapi,13.771,100.6475,RABIKA Coffee,13.770872,100.647133,Coffee Shop
4,Bang Kapi,13.771,100.6475,Ko Boo Rod Sing (โกบู๊รสซิ่ง),13.774971,100.648642,Noodle House


In [9]:
print('There are {} uniques categories.'.format(len(bangkok_venues['Venue Category'].unique())))

There are 163 uniques categories.


In [10]:
bangkok_onehot = pd.get_dummies(bangkok_venues[['Venue Category']], prefix="", prefix_sep="")

bangkok_onehot['District'] = bangkok_venues['District'] 

fixed_columns = [bangkok_onehot.columns[-1]] + list(bangkok_onehot.columns[:-1])
bangkok_onehot = bangkok_onehot[fixed_columns]

bangkok_onehot.head()

,District,Accessories Store,American Restaurant,Art Gallery,Art Museum,Asian Restaurant,Auto Garage,BBQ Joint,Badminton Court,Bagel Shop,...,Theme Park Ride / Attraction,Train Station,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Warehouse Store,Water Park,Wine Bar,Women's Store,Yoga Studio
0,Bang Bon,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Bang Kapi,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Bang Kapi,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Bang Kapi,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Bang Kapi,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
bangkok_grouped = bangkok_onehot.groupby('District').mean().reset_index()
bangkok_grouped

,District,Accessories Store,American Restaurant,Art Gallery,Art Museum,Asian Restaurant,Auto Garage,BBQ Joint,Badminton Court,Bagel Shop,...,Theme Park Ride / Attraction,Train Station,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Warehouse Store,Water Park,Wine Bar,Women's Store,Yoga Studio
0,Bang Bon,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
1,Bang Kapi,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
2,Bang Khae,0.000000,0.000000,0.000000,0.000000,0.090909,0.090909,0.090909,0.090909,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
3,Bang Khen,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
4,Bang Kho Laem,0.000000,0.000000,0.000000,0.000000,0.125000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
5,Bang Khun Thian,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
6,Bang Na,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
7,Bang Phlat,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
8,Bang Rak,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.031250,0.031250,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.031250,0.000000,0.000000
9,Bang Su,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.090909,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000


In [12]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [13]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['District']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['District'] = bangkok_grouped['District']

for ind in np.arange(bangkok_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(bangkok_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bang Bon,Asian Restaurant,Yoga Studio,Farmers Market,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market,Film Studio,Filipino Restaurant
1,Bang Kapi,Noodle House,Thai Restaurant,Convenience Store,Bowling Alley,Flea Market,Coffee Shop,Som Tum Restaurant,Duty-free Shop,Electronics Store,Dumpling Restaurant
2,Bang Khae,Café,Som Tum Restaurant,Dessert Shop,Asian Restaurant,Auto Garage,BBQ Joint,Badminton Court,Night Market,Chinese Restaurant,Fast Food Restaurant
3,Bang Khen,Noodle House,Convenience Store,Badminton Court,Coffee Shop,Spa,Yoga Studio,Fast Food Restaurant,Food & Drink Shop,Food,Flower Shop
4,Bang Kho Laem,Diner,Convenience Store,Sports Club,Market,Noodle House,Leather Goods Store,Paper / Office Supplies Store,Asian Restaurant,Electronics Store,Duty-free Shop


In [14]:
# set number of clusters
kclusters = 5

bangkok_grouped_clustering = bangkok_grouped.drop('District', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bangkok_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 0, 0, 1, 1, 0, 0, 1, 0, 0])

In [15]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

bangkok_merged = df_usable

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
bangkok_merged = bangkok_merged.join(neighborhoods_venues_sorted.set_index('District'), on='District')

bangkok_merged.head(10) # check the last columns!

,District,District_ID,LAT,LONG,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bang Bon,1050,13.646000,100.37000,3.0,Asian Restaurant,Yoga Studio,Farmers Market,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market,Film Studio,Filipino Restaurant
1,Bang Kapi,1006,13.771000,100.64750,0.0,Noodle House,Thai Restaurant,Convenience Store,Bowling Alley,Flea Market,Coffee Shop,Som Tum Restaurant,Duty-free Shop,Electronics Store,Dumpling Restaurant
2,Bang Khae,1040,13.710500,100.39750,0.0,Café,Som Tum Restaurant,Dessert Shop,Asian Restaurant,Auto Garage,BBQ Joint,Badminton Court,Night Market,Chinese Restaurant,Fast Food Restaurant
3,Bang Khen,1005,13.867000,100.62800,1.0,Noodle House,Convenience Store,Badminton Court,Coffee Shop,Spa,Yoga Studio,Fast Food Restaurant,Food & Drink Shop,Food,Flower Shop
4,Bang Kho Laem,1031,13.698667,100.50600,1.0,Diner,Convenience Store,Sports Club,Market,Noodle House,Leather Goods Store,Paper / Office Supplies Store,Asian Restaurant,Electronics Store,Duty-free Shop
5,Bang Khun Thian,1021,13.580000,100.41450,0.0,Dim Sum Restaurant,Basketball Court,American Restaurant,Soccer Field,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market,Film Studio
6,Bang Na,1047,13.672000,100.61600,0.0,Thai Restaurant,Soccer Field,Bus Station,Hotel,Chinese Restaurant,Farmers Market,Flower Shop,Flea Market,Film Studio,Filipino Restaurant
7,Bang Phlat,1025,13.787000,100.49325,1.0,Convenience Store,Noodle House,Market,Food Truck,Yoga Studio,Filipino Restaurant,Food & Drink Shop,Food,Flower Shop,Flea Market
8,Bang Rak,1004,13.728000,100.52320,0.0,Hotel,Chinese Restaurant,Café,Thai Restaurant,Noodle House,Bakery,Dance Studio,Seafood Restaurant,Burger Joint,Malay Restaurant
9,Bang Su,1029,13.820000,100.52900,0.0,Café,Food Truck,Train Station,Pub,Market,Night Market,Noodle House,Thai Restaurant,Bakery,Convenience Store


In [16]:
bangkok_merged.dropna(inplace=True)
bangkok_merged

,District,District_ID,LAT,LONG,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bang Bon,1050,13.646000,100.370000,3.0,Asian Restaurant,Yoga Studio,Farmers Market,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market,Film Studio,Filipino Restaurant
1,Bang Kapi,1006,13.771000,100.647500,0.0,Noodle House,Thai Restaurant,Convenience Store,Bowling Alley,Flea Market,Coffee Shop,Som Tum Restaurant,Duty-free Shop,Electronics Store,Dumpling Restaurant
2,Bang Khae,1040,13.710500,100.397500,0.0,Café,Som Tum Restaurant,Dessert Shop,Asian Restaurant,Auto Garage,BBQ Joint,Badminton Court,Night Market,Chinese Restaurant,Fast Food Restaurant
3,Bang Khen,1005,13.867000,100.628000,1.0,Noodle House,Convenience Store,Badminton Court,Coffee Shop,Spa,Yoga Studio,Fast Food Restaurant,Food & Drink Shop,Food,Flower Shop
4,Bang Kho Laem,1031,13.698667,100.506000,1.0,Diner,Convenience Store,Sports Club,Market,Noodle House,Leather Goods Store,Paper / Office Supplies Store,Asian Restaurant,Electronics Store,Duty-free Shop
5,Bang Khun Thian,1021,13.580000,100.414500,0.0,Dim Sum Restaurant,Basketball Court,American Restaurant,Soccer Field,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market,Film Studio
6,Bang Na,1047,13.672000,100.616000,0.0,Thai Restaurant,Soccer Field,Bus Station,Hotel,Chinese Restaurant,Farmers Market,Flower Shop,Flea Market,Film Studio,Filipino Restaurant
7,Bang Phlat,1025,13.787000,100.493250,1.0,Convenience Store,Noodle House,Market,Food Truck,Yoga Studio,Filipino Restaurant,Food & Drink Shop,Food,Flower Shop,Flea Market
8,Bang Rak,1004,13.728000,100.523200,0.0,Hotel,Chinese Restaurant,Café,Thai Restaurant,Noodle House,Bakery,Dance Studio,Seafood Restaurant,Burger Joint,Malay Restaurant
9,Bang Su,1029,13.820000,100.529000,0.0,Café,Food Truck,Train Station,Pub,Market,Night Market,Noodle House,Thai Restaurant,Bakery,Convenience Store


In [17]:
latitude=13.736717
longitude=100.523186
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(bangkok_merged['LAT'], bangkok_merged['LONG'], bangkok_merged['District'], bangkok_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [18]:
bangkok_merged.loc[bangkok_merged['Cluster Labels'] == 0, bangkok_merged.columns[[1] + list(range(5, bangkok_merged.shape[1]))]]

,District_ID,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,1006,Noodle House,Thai Restaurant,Convenience Store,Bowling Alley,Flea Market,Coffee Shop,Som Tum Restaurant,Duty-free Shop,Electronics Store,Dumpling Restaurant
2,1040,Café,Som Tum Restaurant,Dessert Shop,Asian Restaurant,Auto Garage,BBQ Joint,Badminton Court,Night Market,Chinese Restaurant,Fast Food Restaurant
5,1021,Dim Sum Restaurant,Basketball Court,American Restaurant,Soccer Field,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market,Film Studio
6,1047,Thai Restaurant,Soccer Field,Bus Station,Hotel,Chinese Restaurant,Farmers Market,Flower Shop,Flea Market,Film Studio,Filipino Restaurant
8,1004,Hotel,Chinese Restaurant,Café,Thai Restaurant,Noodle House,Bakery,Dance Studio,Seafood Restaurant,Burger Joint,Malay Restaurant
9,1029,Café,Food Truck,Train Station,Pub,Market,Night Market,Noodle House,Thai Restaurant,Bakery,Convenience Store
10,1020,Convenience Store,Market,Coffee Shop,Restaurant,Som Tum Restaurant,Ice Cream Shop,Food Court,Fast Food Restaurant,Noodle House,Canal
12,1027,Noodle House,Intersection,Coffee Shop,Shopping Mall,Café,Thai Restaurant,Som Tum Restaurant,Sporting Goods Shop,Dumpling Restaurant,Hotpot Restaurant
13,1030,Coffee Shop,Flea Market,Noodle House,Thai Restaurant,Japanese Restaurant,Fast Food Restaurant,Beer Bar,Ramen Restaurant,Multiplex,Shabu-Shabu Restaurant
14,1035,Thai Restaurant,Noodle House,Café,Flea Market,Yoga Studio,Food,Flower Shop,Film Studio,Filipino Restaurant,Fast Food Restaurant


In [19]:
bangkok_merged.loc[bangkok_merged['Cluster Labels'] == 1, bangkok_merged.columns[[1] + list(range(5, bangkok_merged.shape[1]))]]

,District_ID,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,1005,Noodle House,Convenience Store,Badminton Court,Coffee Shop,Spa,Yoga Studio,Fast Food Restaurant,Food & Drink Shop,Food,Flower Shop
4,1031,Diner,Convenience Store,Sports Club,Market,Noodle House,Leather Goods Store,Paper / Office Supplies Store,Asian Restaurant,Electronics Store,Duty-free Shop
7,1025,Convenience Store,Noodle House,Market,Food Truck,Yoga Studio,Filipino Restaurant,Food & Drink Shop,Food,Flower Shop,Flea Market
11,1016,Intersection,Convenience Store,Market,Chinese Restaurant,Fast Food Restaurant,Food & Drink Shop,Food,Flower Shop,Flea Market,Film Studio
22,1033,Food,Convenience Store,Badminton Court,Spa,Yoga Studio,Fast Food Restaurant,Food Court,Food & Drink Shop,Flower Shop,Flea Market
28,1023,Lighthouse,Noodle House,Convenience Store,Bagel Shop,Yoga Studio,Food & Drink Shop,Food,Flower Shop,Flea Market,Film Studio
34,1008,Noodle House,Asian Restaurant,Convenience Store,Hotpot Restaurant,Restaurant,Japanese Curry Restaurant,Ice Cream Shop,Chinese Restaurant,Satay Restaurant,Coffee Shop
45,1015,Noodle House,Asian Restaurant,Bus Stop,Train Station,Convenience Store,Farmers Market,Yoga Studio,Fast Food Restaurant,Food,Flower Shop


In [20]:
bangkok_merged.loc[bangkok_merged['Cluster Labels'] == 2, bangkok_merged.columns[[1] + list(range(5, bangkok_merged.shape[1]))]]

,District_ID,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
42,1034,Convenience Store,Yoga Studio,Farmers Market,Food & Drink Shop,Food,Flower Shop,Flea Market,Film Studio,Filipino Restaurant,Fast Food Restaurant


In [21]:
bangkok_merged.loc[bangkok_merged['Cluster Labels'] == 3, bangkok_merged.columns[[1] + list(range(5, bangkok_merged.shape[1]))]]

,District_ID,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1050,Asian Restaurant,Yoga Studio,Farmers Market,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market,Film Studio,Filipino Restaurant


In [22]:
bangkok_merged.loc[bangkok_merged['Cluster Labels'] == 4, bangkok_merged.columns[[1] + list(range(5, bangkok_merged.shape[1]))]]

,District_ID,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,1009,Thai Restaurant,Yoga Studio,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market,Film Studio,Filipino Restaurant,Fast Food Restaurant
